In [1]:
# this was to try to set up traffic before taking images - not working - cars not moving in autopilot
# adding traffic of X cars to the town
#setting all cars on autopilit
#spawning a ghero hehicle and attaching camrea to it

import glob
import os
import sys
import time

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import time
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(60)

#optional
client.load_world('Town03')
# time.sleep(5)
# world = client.get_world()
# traffic_manager = client.get_trafficmanager(8000)
# tm_port = traffic_manager.get_port()
#settings = world.get_settings()
#settings.synchronous_mode = True
#settings.fixed_delta_seconds = 0.1
#settings.no_rendering_mode = False
# traffic_manager.set_synchronous_mode(True)
# traffic_manager.global_percentage_speed_difference(70)
#world.apply_settings(settings)
# time.sleep(5)

In [11]:
#world.apply_settings(settings)

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

x = 10

vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
spawn_points = world.get_map().get_spawn_points()
for i in range(0,x):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))
time.sleep(2)

vehicle_list = world.get_actors().filter('*vehicle*')
for v in vehicle_list:
     v.set_autopilot(True,tm_port)



In [12]:
vehicle_bp = world.get_blueprint_library().filter('*model3*')

vehicle= None
while vehicle is None:
    vehicle = world.try_spawn_actor(vehicle_bp[0], random.choice(spawn_points))
    time.sleep(2)
vehicle.set_autopilot(True)

#Tesla m3 camera setup aligned to real footage 

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '480')
camera_bp.set_attribute('fov', '90')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

# display the camera
while True:
      
    # Dispaly with imshow
    cv2.imshow('All cameras',camera_data['image'])
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [4]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
spectator = world.get_spectator()
spectator.set_transform(start_point)

In [3]:
client.load_world('Town03')

In [7]:
# get colour from where mouse is - ChatGPT
import cv2

# Function to get RGB values at a specific point
def get_rgb_values(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        # Get the RGB values at the clicked point
        b, g, r = img[y, x]
        print(f"RGB values at ({x}, {y}): R={r}, G={g}, B={b}")

# Load an image using OpenCV
img = cv2.imread('C:/SelfDrive/Semantic segmentation/out_sem/sem/1702966369463358200.png')  # Replace 'your_image_path.jpg' with the path to your image

# Create a window and set the mouse callback function
cv2.namedWindow('image')
cv2.setMouseCallback('image', get_rgb_values)

while True:
    cv2.imshow('image', img)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()